In [1]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

DEBUG:__main__:hello


In [2]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# 2022年3月10日～

# 組み合わせも出るとベースモデルで同じMAPEになってしまう理由の調査

## 調査メモ

* processes リストが 1-128 の場合は上記の問題が発生するが、1-256 の場合は発生しない

In [3]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D"]
processes = [2, 4, 8, 16, 32, 64, 128]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    # "modelIp": "反比例",
    # "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    # "modelIpAndLin": "反比例＆線形",
    # "modelIpAndLog": "反比例＆対数",
    # "modelLogAndLin": "対数＆線形",
    # "modelLogAndIp": "対数＆反比例",
    # "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る"
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年3月16日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

rawDF = rawDF[rawDF["functionName"] == "CONJ_GRAD"]

exp_var = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)
df = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

pd.options.display.precision = 15
print(df)

print(
    df.style.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
        label=f"{date}allModelsMAPEin{benchmark_name.upper()}",
    )
)
pd.options.display.precision = 6

                           線形              線形＆反比例               線形＆対数 最適モデル
関数名                                                                        
CONJ_GRAD  16.051673712330807  16.051673714205847  16.051673714205901    線形
\begin{table}
\caption{ベンチマークプログラムCGにおける各モデルでのMAPE}
\label{2022年3月16日allModelsMAPEinCG}
\begin{tabular}{lrrrl}
 & 線形 & 線形＆反比例 & 線形＆対数 & 最適モデル \\
関数名 &  &  &  &  \\
CONJ_GRAD & 16.051674 & 16.051674 & 16.051674 & 線形 \\
\end{tabular}
\end{table}



In [4]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年2月2日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

rawDF_CONJ_GRAD = rawDF[rawDF["functionName"] == "CONJ_GRAD"]
rawDF_CONJ_GRAD

# returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
#     rawDF=rawDF,
#     exp_var_list=exp_var,
#     res_var_list=res_var,
#     model_name_list=model_names_list,
# )
# df = returned_MAPE_table.rename(columns=dict_column_names_JP)
# df = df.set_index("関数名")
# df = df.sort_index(axis=0)
# df = df.sort_index(axis=1)

# df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

# print(
#     df.to_latex(
#         caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
#         label=f"{date}everyModelsMAPEtableIn{benchmark_name.upper()}",
#     )
# )

,functionName,functionCallNum,benchmarkName,benchmarkClass,process,intBenchmarkClass,na,nonzer,niter,shift
5,CONJ_GRAD,16.0,cg,A,2,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,4,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,8,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,16,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,32,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,64,1,14000,11,15,20
5,CONJ_GRAD,16.0,cg,A,128,1,14000,11,15,20
6,CONJ_GRAD,16.0,cg,A,256,1,14000,11,15,20
2,CONJ_GRAD,31.0,cg,B,2,4,75000,13,75,60
2,CONJ_GRAD,46.0,cg,B,4,4,75000,13,75,60


# （2022年2月23日～2022年3月7日）

## ✅新モデルの実装

# (2022年3月7日～2022年3月9日)

## ✅新モデルを含めたMAPE表の作成

In [5]:
benchmark_name = "cg"

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

dict_column_names_JP = {
    "functionName": "関数名",
    "modelLin": "線形",
    "modelIp": "反比例",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る"
}

model_names_list_JP = list(dict_column_names_JP.values())
model_names_list_JP.remove("関数名")

model_names_list = list(dict_column_names_JP.keys())
model_names_list.remove("functionName")

date = "2022年3月9日"

# コア数と問題サイズの両方を振る

rawDF = return_rawDF_with_init_param(
    benchmark_name=benchmark_name,
    classes=classes,
    processes=processes,
    csv_dir_path="./csv_files/",
)

exp_var = rawDF.columns.tolist()
for element_be_removed in [
    "functionName",
    "functionCallNum",
    "intBenchmarkClass",
    "benchmarkName",
    "benchmarkClass",
]:
    exp_var.remove(element_be_removed)
res_var = ["functionCallNum"]

print(f"exp_var = {exp_var}")

returned_MAPE_table = return_MAPE_Table_DF_from_rawDF(
    rawDF=rawDF,
    exp_var_list=exp_var,
    res_var_list=res_var,
    model_name_list=model_names_list,
)
df = returned_MAPE_table.rename(columns=dict_column_names_JP)
df = df.set_index("関数名")
df = df.sort_index(axis=0)
df = df.sort_index(axis=1)

df = addLowestMAPEsModelNameColumn(df, version=2, model_name_list=model_names_list_JP)

print(
    df.style.to_latex(
        caption=f"ベンチマークプログラム{benchmark_name.upper()}における各モデルでのMAPE",
        label=f"{date}allModelsMAPEin{benchmark_name.upper()}",
    )
)

exp_var = ['process', 'na', 'nonzer', 'niter', 'shift']


/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estima

\begin{table}
\caption{ベンチマークプログラムCGにおける各モデルでのMAPE}
\label{2022年3月9日allModelsMAPEinCG}
\begin{tabular}{lrrrrrrrrrrl}
 & プロセス数を問題サイズで割る & 反比例 & 反比例＆対数 & 反比例＆線形 & 対数 & 対数＆反比例 & 対数＆線形 & 線形 & 線形＆反比例 & 線形＆対数 & 最適モデル \\
関数名 &  &  &  &  &  &  &  &  &  &  &  \\
.TAU_application & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 線形 \\
ALLOC_SPACE & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 線形 \\
CG & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 0.000000 & 線形 \\
CONJ_GRAD & 37.550135 & 13.876670 & 13.732936 & 14.498431 & 14.804389 & 14.717362 & 15.248570 & 13.161174 & 12.412559 & 12.755412 & 線形＆反比例 \\
ICNVRT & 159.614199 & 662.481358 & 2504.274677 & 894.574028 & 2443.699386 & 3811.490854 & 815.789372 & 502.123230 & 6621.011052 & 2283.060136 & プロセス数を問題サイズで割る \\
INITIALIZE_MPI & 0.000000 & 0.000000 & 0.000000 & 0.0000